In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy.stats import fisher_exact
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from random import randint
import pandas as pd
import numpy as np
import random
from sklearn import datasets, linear_model
from __future__ import division
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
import pydot
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
data = pd.read_csv('new_file7.csv')

* Actual time taken is a proxy variable for delivered to plan variable
* So, the problem becomes regression problem instead of classification problem

In [119]:
temp =(data.Planned_days - data.Actual_timetaken)
temp1 = temp.apply(lambda x: round(x,2))
tab = pd.crosstab(temp1>=0,data.delivered_to_plan)
print(tab)

delivered_to_plan   FAIL   PASS
row_0                          
False              11591      0
True                   0  98523


* The number of late deliveries are 11591

* Two types of Performance measure: 1. Number of delivery fails 2. How much time saved by new method 
* Change the delivery date of the customer based on this model

In [3]:
data.returned.fillna('N',inplace = True)
data.backorder.fillna('N',inplace = True)
country = pd.get_dummies(data['country'],prefix='country',drop_first=True)
shipping_method = pd.get_dummies(data['shipping_method'],prefix='shipping',drop_first=True)
product_category = pd.get_dummies(data['product_category'],prefix='product_category',drop_first=True)
facility = pd.get_dummies(data['facility'],prefix='facility',drop_first=True)
on_sale =  pd.get_dummies(data['on_sale'],prefix='onsale',drop_first=True)
Returned =  pd.get_dummies(data['returned'],prefix='Returned',drop_first=True)
Backorder = pd.get_dummies(data['backorder'],prefix='Returned',drop_first=True)

* One hot encoding for all categorical variables

In [4]:
data_new = pd.concat([country,shipping_method,facility,product_category,on_sale,Returned,Backorder,data['units_per_order'],
          data['transit_days'],data['source'],data['make'],data['Exact_deliver_time'],data['delivered_to_plan'],data['Actual_timetaken'],data['Planned_days']],axis=1)
data_F = data_new[data_new.delivered_to_plan == 'FAIL']
data_P = data_new[data_new.delivered_to_plan == 'PASS']

* Traditional train test split doesn't work because of two reasons: 1. Created user defined performance measure to evaluate all the models 2. It is imbalanced dataset (The number of late deliveries are less compared to the normal deliveries)

In [5]:
temp_test = random.sample(range(len(data_F)), round(len(data_F)*0.15))
temp_train = list(set(np.arange(len(data_F))) - set(random.sample(range(len(data_F)), round(len(data_F)*0.15))))
test= data_F.iloc[temp_test,:]
train = data_F.iloc[temp_train,:]

temp_test1 = random.sample(range(len(data_P)), round(len(data_P)*0.15))
temp_train1 = list(set(np.arange(len(data_P))) - set(random.sample(range(len(data_P)), round(len(data_P)*0.15))))
test1= data_P.iloc[temp_test1,:]
train1 = data_P.iloc[temp_train1,:]

test_data = pd.concat([test,test1],axis=0)
train_data = pd.concat([train,train1],axis=0)

In [6]:
X = train_data.iloc[:,:-6]
y_s = train_data.source
y_m = train_data.make
y_d = train_data.Exact_deliver_time

X_test = test_data.iloc[:,:-6]
y_test_s = test_data.source
y_test_m = test_data.make
y_test_d = test_data.Exact_deliver_time

* Performance measure to capture: Tardy/Late deliveries PM
* TL Performance measure for the heuristics used by the company is 23985 days

In [9]:
print(sum(abs(test_data.Planned_days - test_data.Actual_timetaken)))
print(performance_measure(test_data.Actual_timetaken,test_data.Planned_days))

23985.0
(38333.0, 6202)


* Defined a new performance measure because the over estimation of the predicted time doesn't lead to the late delivery, but underestimation does.
* The number of times fail to complete the source task before the deadline given by simple linear regression is 3347
* The number of times fail to complete the make task before the deadline given by simple linear regression is 3277
* The number of times fail to complete the deliver task before the deadline given by simple linear regression is 6488
* The number of times fail to deliver before the deadline given by simple linear regression is 10202 

In [7]:
def performance_measure(y_test,prediction):
    temp = y_test - prediction 
    temp1 = temp.apply(lambda x: round(x,2))
    Ind = temp1 >= 0
    error = 5*(sum(abs(y_test[Ind] - prediction[Ind]))) + sum(abs(y_test[~Ind] - prediction[~Ind]))
    num_fails = sum(Ind) 
    return error,num_fails

## Linear Regression

In [17]:
regressor1 = LinearRegression()
regressor1.fit(X, y_s)
prediction1 = regressor1.predict(X_test)
print(performance_measure(y_test_s,prediction1))

(16129572.418489572, 3262)


In [18]:
regressor2 = LinearRegression()
regressor2.fit(X, y_m)
prediction2 = regressor2.predict(X_test)
performance_measure(y_test_m,prediction2)

(528928.09926459671, 6613)

In [19]:
regressor3 = LinearRegression()
regressor3.fit(X, y_d)
prediction3 = regressor3.predict(X_test)
performance_measure(y_test_d,prediction3)

(52455.129440782322, 6755)

In [22]:
Actual_timetaken = (y_test_s/1440) + (y_test_m/24) + y_test_d
prediction_actual_timetaken = (prediction1/1440) + (prediction2/24) + prediction3
print(performance_measure(Actual_timetaken,prediction_actual_timetaken))
print(sum(abs(test_data.Planned_days - prediction_actual_timetaken)))

(57888.56077902345, 7164)
22860.2813852


In [23]:
print(regressor1.coef_)
print(regressor2.coef_)
print(regressor3.coef_)

[  2.17539141e+13   1.26413867e+03   1.78619455e+14   1.78619455e+14
   1.78619455e+14   1.21039062e+02   1.09460938e+02   6.06406250e+01
   2.17539141e+13   2.17539141e+13   2.17539141e+13  -1.56865541e+14
  -1.56865541e+14   1.66054688e+01   7.17968750e+00   5.29765625e+01
  -1.13725586e+01   1.22309375e+03   2.69921875e+00   6.25000000e+00]
[  3.86542885e+11  -3.64166260e+00  -9.08621933e+12  -9.08621933e+12
  -9.08621933e+12  -1.42211914e-02   2.09075928e-01  -5.05859375e+00
   3.86542885e+11   3.86542885e+11   3.86542885e+11   9.47276221e+12
   9.47276221e+12  -5.48339844e-01  -1.92138672e-01  -3.75976562e-02
   2.99255371e-01  -2.52783203e+00  -3.49121094e-02  -8.39843750e-02]
[ -2.23407334e+11  -1.22833252e-01  -1.75801966e+12  -1.75801966e+12
  -1.75801966e+12   1.13119507e+00   6.22070312e-01   1.44195557e-02
  -2.23407334e+11  -2.23407334e+11  -2.23407334e+11   1.53461232e+12
   1.53461232e+12   1.12915039e-02   2.27355957e-02  -3.40576172e-02
   8.07533264e-02  -1.29882812e-

* Polynomial features: Included only interaction terms since most of the features are binary features
* The number of times fail to complete the source task before the deadline given by simple linear regression is 3062
* The number of times fail to complete the make task before the deadline given by simple linear regression is 6620
* The number of times fail to complete the deliver task before the deadline given by simple linear regression is 6483
* The number of times fail to deliver before the deadline given by simple linear regression is 7163 

## Polynomial regression

In [24]:
poly_model1 = make_pipeline(PolynomialFeatures(interaction_only=True),
                           LinearRegression())
poly_model1.fit(X, y_s)
prediction1 = poly_model1.predict(X_test)
performance_measure(y_test_s,prediction1)

(15963091.608333306, 2927)

In [25]:
poly_model2 = make_pipeline(PolynomialFeatures(interaction_only=True),
                           LinearRegression())
poly_model2.fit(X, y_m)
prediction2 = poly_model2.predict(X_test)
performance_measure(y_test_m,prediction2)

(527231.04736111325, 6598)

In [26]:
poly_model3 = make_pipeline(PolynomialFeatures(interaction_only=True),
                           LinearRegression())
poly_model3.fit(X, y_d)
prediction3 = poly_model3.predict(X_test)
performance_measure(y_test_d,prediction3)

(51259.996035156233, 6498)

In [30]:
prediction_actual_timetaken = (prediction1/1440) + (prediction2/24) + prediction3
print(performance_measure(Actual_timetaken,prediction_actual_timetaken))
print(sum(abs(test_data.Planned_days - prediction_actual_timetaken)))

(56770.228630642261, 7160)
22802.9557321


* Coefficients of polynomial regression

In [28]:
poly_model1.steps[1][1].coef_

array([  4.97393967e+10,  -1.11464563e+15,  -2.73347957e+15,
        -4.38326510e+15,   2.31105177e+15,  -5.59914670e+15,
         2.42371405e+14,   3.15683493e+14,  -7.35441349e+14,
        -1.36470453e+14,  -1.07624896e+15,  -9.61690227e+12,
         5.03289071e+14,   1.34545880e+15,   2.30798876e+13,
        -9.55033003e+14,  -3.45739309e+14,   5.78140234e+13,
        -1.26852688e+14,   3.75744258e+13,  -7.38424784e+14,
        -7.82302250e+12,   6.48422645e+13,   8.88897819e+13,
         4.24619966e+13,   4.06725581e+12,   1.36888053e+14,
        -1.00814410e+15,   2.47576885e+12,  -1.35853114e+14,
        -2.77014946e+13,  -2.22642792e+14,  -2.70363461e+14,
        -2.30798876e+13,   9.55033003e+14,   3.45739309e+14,
        -5.78140234e+13,   1.26852688e+14,  -3.75744258e+13,
         1.70731124e+15,   5.51954403e+12,  -4.42809296e+13,
        -6.95522120e+13,   1.69331995e+15,   1.52359804e+15,
         3.33821086e+14,   2.70103028e+14,   1.20988154e+15,
         1.43249477e+14,

## Linear Regression using Prediction Interval
* t_value is a hyperparameter
* The number of times fail to complete the source task before the deadline given by simple linear regression is 1526
* The number of times fail to complete the make task before the deadline given by simple linear regression is 5255
* The number of times fail to complete the deliver task before the deadline given by simple linear regression is 5237
* The number of times fail to deliver before the deadline given by simple linear regression is 4868 

In [94]:
class LRPI:
    def __init__(self, normalize=False, n_jobs=1):
        self.normalize = normalize
        self.n_jobs = n_jobs
        self.LR = linear_model.LinearRegression(normalize=self.normalize, n_jobs= self.n_jobs)
        
    def fit(self, X_train, y_train):
        self.X_train = pd.DataFrame(X_train.values)
        self.y_train = pd.DataFrame(y_train.values)
        
        self.LR.fit(self.X_train, self.y_train)
        X_train_fit = self.LR.predict(self.X_train)
        self.MSE = np.power(self.y_train.subtract(X_train_fit), 2).sum(axis=0) / (self.X_train.shape[0] - self.X_train.shape[1] - 1)
        self.X_train.loc[:, 'const_one'] = 1
        self.XTX_inv = np.linalg.inv(np.dot(np.transpose(self.X_train.values) , self.X_train.values))
        
    def predict(self, X_test,t_value=0.2):
        self.t_value = t_value
        self.X_test = pd.DataFrame(X_test.values)
        self.pred = self.LR.predict(self.X_test)
        self.X_test.loc[: , 'const_one'] =1
        SE = [np.dot(np.transpose(self.X_test.values[i]) , np.dot(self.XTX_inv, self.X_test.values[i]) ) for i in range(len(self.X_test)) ]
        results = pd.DataFrame(self.pred , columns=['Pred'])
        
        results.loc[:,"lower"] = results['Pred'].subtract((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        results.loc[:,"upper"] = results['Pred'].add((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        
        return results

In [32]:
model1 = LRPI()
model1.fit(X, y_s)
results1 = model1.predict(X_test,0.2)
y_test_s.reset_index(drop = True, inplace = True)

In [33]:
performance_measure(y_test_s,results1.upper)

(17140074.128086239, 1517)

In [34]:
model2 = LRPI()
model2.fit(X, y_m)
results2 = model2.predict(X_test,0.2)
y_test_m.reset_index(drop = True, inplace = True)
performance_measure(y_test_m,results2.upper)

(472587.74420984735, 5215)

In [35]:
model3 = LRPI()
model3.fit(X, y_d)
results3 = model3.predict(X_test,0.2)
y_test_d.reset_index(drop = True, inplace = True)
performance_measure(y_test_d,results3.upper)

(46537.72863496044, 5244)

In [54]:
prediction_actual_timetaken = (results1.upper/1440) + (results2.upper/24) + results3.upper
Actual_timetaken.reset_index(drop = True, inplace = True)
print(performance_measure(Actual_timetaken,prediction_actual_timetaken))
test_planned = test_data.Planned_days
test_planned.reset_index(drop = True, inplace = True)
print(sum(abs(test_planned - prediction_actual_timetaken)))

(46106.702398473062, 4868)
19654.6087016


## Cross-Validation

In [105]:
t_values = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,0.99]
Actual_timetaken = (y_test_s/1440) + (y_test_m/24) + y_test_d
y_test_s.reset_index(drop = True, inplace = True)
y_test_m.reset_index(drop = True, inplace = True)
y_test_d.reset_index(drop = True, inplace = True)
Actual_timetaken.reset_index(drop = True, inplace = True)
model1 = LRPI()
time_vec = []
fails_vec = []
for i in range(len(t_values)):
    model1.fit(X, y_s)
    results1 = model1.predict(X_test,t_values[i])
    time_s, fails_s = performance_measure(y_test_s,results1.upper)
    model1.fit(X, y_m)
    results2 = model1.predict(X_test,t_values[i])
    time_m, fails_m = performance_measure(y_test_m,results2.upper)
    model1.fit(X, y_d)
    results3 = model1.predict(X_test,t_values[i])
    time_d, fails_d = performance_measure(y_test_d,results3.upper)
    prediction_actual_timetaken = (results1.upper/1440) + (results2.upper/24) + results3.upper
    time, fails = performance_measure(Actual_timetaken,prediction_actual_timetaken)
    time_vec.append(time)
    fails_vec.append(fails)

In [108]:
plt.plot(t_values,time_vec)
plt.xlabel('t-values')
plt.ylabel('Performance measure')
plt.title('LRPI')
plt.savefig('lrpi.jpeg',dpi=300)

In [97]:
test_planned = test_data.Planned_days
test_planned.reset_index(drop = True, inplace = True)
index_min = np.argmin(time_vec)
print(fails_vec[index_min])
print(min(time_vec))
print(sum(abs(test_planned - prediction_actual_timetaken)))

2639
43057.6772404
35953.3007746


## Random Forest

In [100]:
num_estimators = [50,100,150,200,250,300,350,400,450,500,550,600,650]
Actual_timetaken = (y_test_s/1440) + (y_test_m/24) + y_test_d
y_test_s.reset_index(drop = True, inplace = True)
y_test_m.reset_index(drop = True, inplace = True)
y_test_d.reset_index(drop = True, inplace = True)
Actual_timetaken.reset_index(drop = True, inplace = True)
time_vec = []
fails_vec = []
for i in range(len(num_estimators)):
    model1 = RandomForestRegressor(n_estimators = num_estimators[i], n_jobs=-1, random_state=42)
    print(i)
    model1.fit(X, y_s)
    results1 = model1.predict(X_test)
    time_s, fails_s = performance_measure(y_test_s,results1)
    model1.fit(X, y_m)
    results2 = model1.predict(X_test)
    time_m, fails_m = performance_measure(y_test_m,results2)
    model1.fit(X, y_d)
    results3 = model1.predict(X_test)
    time_d, fails_d = performance_measure(y_test_d,results3)
    prediction_actual_timetaken = (results1/1440) + (results2/24) + results3
    time, fails = performance_measure(Actual_timetaken,prediction_actual_timetaken)
    time_vec.append(time)
    fails_vec.append(fails)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [103]:
plt.plot(num_estimators,time_vec)
plt.xlabel('n_estimators')
plt.ylabel('Performance measure')
plt.title('Random Forest')
plt.savefig('randomforest.jpeg',dpi=300)

In [27]:
test_planned = test_data.Planned_days
test_planned.reset_index(drop = True, inplace = True)
index_min = np.argmin(time_vec)
print(fails_vec[index_min])
print(min(time_vec))
print(sum(abs(test_planned - prediction_actual_timetaken)))

7160
55909.3696949
22972.769238


In [69]:
num_estimators = [100,200,300,400,500,600,700,800,900,1000,1500,2000,2500,3000,3500,4000]
Actual_timetaken = (y_test_s/1440) + (y_test_m/24) + y_test_d
y_test_s.reset_index(drop = True, inplace = True)
y_test_m.reset_index(drop = True, inplace = True)
y_test_d.reset_index(drop = True, inplace = True)
Actual_timetaken.reset_index(drop = True, inplace = True)
time_vec = []
fails_vec = []
for i in range(len(num_estimators)):
    model1 = GradientBoostingRegressor(max_depth=2, n_estimators=num_estimators[i], learning_rate=0.1, random_state=42)
    print(i)
    model1.fit(X, y_s)
    results1 = model1.predict(X_test)
    time_s, fails_s = performance_measure(y_test_s,results1)
    model1.fit(X, y_m)
    results2 = model1.predict(X_test)
    time_m, fails_m = performance_measure(y_test_m,results2)
    model1.fit(X, y_d)
    results3 = model1.predict(X_test)
    time_d, fails_d = performance_measure(y_test_d,results3)
    prediction_actual_timetaken = (results1/1440) + (results2/24) + results3
    time, fails = performance_measure(Actual_timetaken,prediction_actual_timetaken)
    time_vec.append(time)
    fails_vec.append(fails)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [89]:
plt.plot(num_estimators,time_vec)
plt.xlabel('n_estimators')
plt.ylabel('Performance measure')
plt.title('Gradient Boosting')
plt.savefig('gradient.jpeg',dpi=300)

In [71]:
index_min = np.argmin(time_vec)
print(fails_vec[index_min])
print(min(time_vec))
print(sum(abs(test_planned - prediction_actual_timetaken)))

7126
56523.3887251
22782.5800618
